In [ ]:
import pickle
import numpy as np
import torch

In [ ]:
# Put your model path
model = torch.load('model.pt', map_location='cpu')

In [ ]:
with open('test_none_crash.pickle', 'rb') as f:
	none_crash_test_list = pickle.load(f)

with open('test_vulner.pickle', 'rb') as f:
	vulner_test_list = pickle.load(f)
	
# ex) test_list[i]: "../../dataset/data_preprocessed/None-crash\21-12-01-11-07-44_end_extract_drive26\00449.pickle"
test_list = none_crash_test_list + vulner_test_list

In [ ]:
TP, FP, FN, TN = 0, 0, 0, 0
TP_list = list()
FP_list = list()
FN_list = list()
TN_list = list()

print("#Test-data:", len(test_list))

for i in range(len(test_list)):
    with open(test_list[i], "rb") as fr:
        tmp_data = pickle.load(fr)

    test_label = tmp_data['label']
    if test_label == "None-crash":
        test_label = 0
    else:
        test_label = 1

    # Make tensor as input of model
    test_tensor = np.asarray(tmp_data['tensor'])
    test_tensor = np.asarray([test_tensor])
    test_tensor = torch.from_numpy(test_tensor)
    test_tensor = test_tensor.to(torch.float32).cpu()

    pred = model(test_tensor)
    if pred[0][0] > pred[0][1]:
        pred_idx = 0
    elif pred[0][0] < pred[0][1]:
        pred_idx = 1
    else:
        assert pred[0][0] == pred[0][1], 'Same class score'

    if pred_idx == 1 and test_label == 1:
        TP += 1
        TP_list.append(test_list[i])
    elif pred_idx == 1 and test_label == 0:
        FP += 1
        FP_list.append(test_list[i])
    elif pred_idx == 0 and test_label == 1:
        FN += 1
        FN_list.append(test_list[i])
    elif pred_idx == 0 and test_label == 0:
        TN += 1
        TN_list.append(test_list[i])

In [ ]:
acc = (TP+TN)/(TP+TN+FP+FN)
spec = (TN/(FP+TN))
prec = (TP/(TP+FP))
recall = (TP/(TP+FN))

print("TP:", len(TP_list))
print("TN:", len(TN_list))
print("FP:", len(FP_list))
print("FN:", len(FN_list))

print("Accuracy:", acc)
print("Specificity:", spec)
print("Precision:", prec)
print("Recall:", recall)
print("F1 Score:", (2*prec*recall/(prec+recall)))

### TP, TN, FP, FN이 어떤 파일인지 확인하려면 아래를 실행 시키세요.

In [ ]:
print("TP")
for tp in TP_list:
    print(tp)

In [ ]:
print("TN")
for tn in TN_list:
    print(tn)

In [ ]:
print("FP")
for fp in FP_list:
    print(fp)

In [ ]:
print("FN")
for fn in FN_list:
    print(fn)